In [ ]:
from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix
import numpy as np 
import torch
import dgl
from dgl.data import AsNodePredDataset
from ogb.nodeproppred import DglNodePropPredDataset
from dgl.dataloading import DataLoader, NeighborSampler, MultiLayerFullNeighborSampler

In [ ]:
dataset = AsNodePredDataset(DglNodePropPredDataset('ogbn-papers100M',root="/home/bear/workspace/singleGNN/data/dataset"))
g = dataset[0]
GSRC = g.edges()[0]
GDST = g.edges()[1]

In [ ]:
sampler = NeighborSampler([3,3,3])
train_idx = dataset.train_idx.to('cuda')
train_dataloader = DataLoader(g, train_idx, sampler, device='cuda',
                                batch_size=8, shuffle=False,
                                drop_last=False, num_workers=0,
                                use_uva=True)

In [ ]:
for it, (input_nodes, output_nodes, blocks) in enumerate(train_dataloader):
    print(blocks)
    break

# blocks[0].ndata[dgl.NID] 检测源节点ID
blockEdgeId = blocks[2].edata[dgl.EID] # 检测源边ID
blockEdgeId
blockSRC = GSRC[blockEdgeId]
blockDST = GDST[blockEdgeId]
blockEdge = torch.stack((blockSRC, blockDST), dim=0)
blockEdge # 抽取被采样的边

In [ ]:
edges0 = np.fromfile("/home/bear/workspace/single-gnn/tmp/part0.bin",dtype=np.int32)
trainIds = np.fromfile("/home/bear/workspace/single-gnn/tmp/trainIds0.bin",dtype=np.int64)
edges0 = edges0.reshape(-1,2)
edges0 = edges0[120718:]
src = edges0.reshape(-1)[::2]
dst = edges0.reshape(-1)[1::2]

In [ ]:
t = g.has_edges_between(src,dst) # 测试子图正常性
t.numel() == t.sum()

In [ ]:
mask = np.isin(blockEdge, edges0)
mask.reshape(-1).all()